# Classwork 6 

In [ ]:
# %load ../utils/imports.py
%matplotlib inline

import numpy as np
import pandas as pd

from utils import *
from utils.styles import *

In [ ]:
# %load ../utils/plotting.py
import seaborn as sns

from plotly.offline import init_notebook_mode, iplot
import cufflinks as cf

init_notebook_mode()
cf.go_offline()


Work through the following datasets, determining best fits for each data set (predictor value/y value in parens). To better evaluate or improve this process, try including:

* Increase the number of training points N. This might give us a training set with more coverage, and lead to greater accuracy.
* Increase the degree d of the polynomial. This might allow us to more closely fit the training data, and lead to a better result
* Add more features. If we were to, for example, perform a linear regression using x, x√, x−1, or other functions, we might hit on a functional form which can better be mapped to the value of y.

In [ ]:
import statsmodels.api as sm

%matplotlib inline

DATA_DIR = '../data/'

## Predicting stopping distance

In [ ]:
# Load the data
df = pd.read_csv(DATA_DIR + 'cars1920.csv')

In [ ]:
# explore the features
df.info()

In [ ]:
# explore the types of relationship you should model for, linear?
df.iplot(x="speed", y='dist', mode='markers', bestfit=True, shape='spline')

In [ ]:
sns.lmplot("speed", "dist", df, order=1, aspect=2);

In [ ]:
# explore the types of relationship you should model for, second order polynomial?
sns.lmplot("speed", "dist", df, order=2, aspect=2);

In [ ]:
# explore the types of relationship you should model for, third order polynomial?
sns.lmplot("speed", "dist", df, order=3, aspect=2);

Since it is not strictly a linear relationship, we are going to use polynomial features. 
But there are two problems with this.

In [ ]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures

X = [[i] for i in df.speed.values]
y = df.dist


scores = []
for model in [Ridge, Lasso]:
    for j in [0.1,0.5,1,2,5,10,20,100]: # ALPHA
        for i in range(1,6):            # POLY
            est = model(alpha=j)
            poly = PolynomialFeatures(i)
            X_poly = poly.fit_transform(X)
            est.fit(X_poly,y)
            scores.append((model.__name__, j, i, est.score(X_poly,y)))
            
df_scores = pd.DataFrame(scores, columns=['model','alpha','poly','score'])

### Orthogonal Polynomials - Advanced

First, we can generate highly correlated regressors by taking powers of `x`, leading to noisy parameter estimates. The input `x` are evenly space numbers on the interval `[0, 1]`. So `x` and $x^2$ are going to have a correlation over `95%`. Similar with $x^2$ and $x^3$. The solution to this is to use orthogonalized polynomial functions: tranformations of `x` that, when summed, result in polynomial functions, but are orthogonal (therefore uncorrelated) with each other.

Luckily, we can easily calculate these transformations using patsy. The `C(x, Poly)` transform computes orthonormal polynomial functions of `x`, then we’ll extract out various orders of the polynomial. So `Xpoly[:, :2]` selects out the `0th` and 1st order functions, then when summed will give us a first order polynomial (i.e. linear). Similarly `Xpoly[: :4]` gives us the 0th through 3rd order functions, which sum up to a cubic polynomial.

Math review : [Orthogonal Polynomials](http://mathoverflow.net/questions/38864/visualizing-orthogonal-polynomials)

In [ ]:
from patsy import dmatrix

In [ ]:
speed = df.speed.values
dist = df.dist.values
y = dist

speed_poly = dmatrix('C(speed, Poly)')

In [ ]:
speed_poly

In [ ]:
# Only Speed
speed_poly1 = dmatrix('speed', df)

# Only Speed^2
speed_sq = dmatrix('np.power(speed,2)', df)

# Speed + Speed^2
speed_poly2 = dmatrix('speed + np.power(speed,2)')

# Orthoganal Polynnomials for ^3, ^5 and ^25
speed_poly3 = speed_poly[:, :4]
speed_poly5 = speed_poly[:, :6]
speed_poly25 = speed_poly[:, :26]

In [ ]:
Xs = [speed_poly1, speed_sq, speed_poly2, speed_poly3, speed_poly5, speed_poly25]

In [ ]:
speed_poly2[:5]

In [ ]:
# Let's see how closely they are correlated.
pd.DataFrame(np.corrcoef(speed_poly25)).iplot(kind='heatmap',colorscale='spectral')

### Overfitting

The problem we encounter now is how to choose what order polynomial to fit to the data. Any data can be fit well (i.e. have a high $R^2$) if we use a high enough order polynomial. But we will start to over-fit our data; capturing noise specific to our sample, leading to poor predictions on new data. The graph below shows the fits to the data of a straight line, a 3rd-order polynomial, a 5th-order polynomial, and a 25th-order polynomial.

In [ ]:
speed_poly1_pred = sm.OLS(y, speed_poly1).fit().predict()
speed_sq_pred = sm.OLS(y, speed_sq).fit().predict()
speed_poly2_pred = sm.OLS(y, speed_poly2).fit().predict()
speed_poly3_pred = sm.OLS(y, speed_poly3).fit().predict()
speed_poly5_pred = sm.OLS(y, speed_poly5).fit().predict()
speed_poly25_pred = sm.OLS(y, speed_poly25).fit().predict()

In [ ]:
import matplotlib.pyplot as plt

plt.figure()

fig, ax = plt.subplots(3, 2, sharex = True, sharey = True, figsize=(16,9))
fig.subplots_adjust(hspace = 0.0, wspace = 0.0)
fig.suptitle('Polynomial fits to stopping distance', fontsize = 16.0)

# Iterate through panels (a), model predictions (p), and the polynomial 
# degree of the model (d). Plot the data, the predictions, and label
# the graph to indicate the degree used.
preds = [speed_poly1_pred, speed_sq_pred, speed_poly2_pred,
         speed_poly3_pred, speed_poly5_pred, speed_poly25_pred]
orders = ['1', 'sq', '2', '3', '5', '25']

for a, p, d in zip(ax.ravel(), preds, orders):
    a.plot(df.speed.values, df.dist.values, '.', color = 'steelblue', alpha = 0.5)
    a.plot(df.speed.values, p)
    a.text(.5, .95, 'D = ' + d, fontsize = 12,
           verticalalignment = 'top',
           horizontalalignment = 'center',
           transform = a.transAxes)
    a.grid()
    
# Alternate axes that have tick labels to avoid overlap.
plt.setp(fig.axes[2].get_yaxis().get_ticklabels(), visible = False)
plt.setp(fig.axes[3].get_yaxis(), ticks_position = 'right')   
plt.setp(fig.axes[1].get_xaxis(), ticks_position = 'top')
plt.setp(fig.axes[3].get_xaxis().get_ticklabels(), visible = False);

 Notice how the last fit gives us all kinds of degrees of freedom to capture specific datapoints, and the excessive “wiggles” look like we’re fitting to noise.

### Scoring your models

In [ ]:
for order, X in zip(orders, Xs):
    print "%s : %.03f" % (order, sm.OLS(y, X).fit().rsquared)

Without cross-validation - which we'll be covering soon, it would appear that the `sq` model is preferred as that provides us with a smooth line, 

In [ ]:
# The same with our SciKit Learn Models

from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures

X = [[i] for i in df.speed.values]
y = df.dist

scores = []
for model in [Ridge, Lasso]:
    for j in [0.1,0.5,1,2,5,10,20,100]: # ALPHA
        for i in range(1,14):           # POLY
            est = model(alpha=j)
            poly = PolynomialFeatures(i)
            X_poly = poly.fit_transform(X)
            est.fit(X_poly,y)
            scores.append((model.__name__, j, i, est.score(X_poly,y)))
            
df_scores = pd.DataFrame(scores, columns=['model','alpha','poly','score'])

title = '<b>R-Squared for Linear Regression</b><br>With L1 and L2 Regularisation'
yTitle = 'R-Squared'
xTitle = 'Number of Polynominals'

df_scores.groupby(['model','poly']).max().score.unstack(0).iplot(title=title,yTitle=yTitle,xTitle=xTitle)

Now let's try this with cross-validation

In [ ]:
from sklearn.cross_validation import cross_val_score

scores = []
for model in [Ridge, Lasso]:
    for j in [0.1,0.5,1,2,5,10,20,100]: # ALPHA
        for i in range(1,10):           # POLY
            est = model(alpha=j)
            poly = PolynomialFeatures(i)
            X_poly = poly.fit_transform(X)
            
            scores.append((model.__name__, j, i, cross_val_score(est, X_poly, y).mean()))
            
df_scores = pd.DataFrame(scores, columns=['model','alpha','poly','score'])

title = '<b>R-Squared for Linear Regression</b><br>With L1 and L2 Regularisation'
yTitle = 'R-Squared'
xTitle = 'Number of Polynominals'

df_scores.groupby(['model','poly']).max().score.unstack(0).iplot(title=title,yTitle=yTitle,xTitle=xTitle)

## Predicting City and Highway MPG.

In [ ]:
# Load the data
import pandas as pd

DATA_DIR = '../data/'

df = pd.read_csv(DATA_DIR + 'cars93.csv')

### Inspect the Data

In [ ]:
df.info()

In [ ]:
df.Cylinders.value_counts()

In [ ]:
df.Make.head()

In [ ]:
range(5)

In [ ]:
pd.get_dummies(df.Manufacturer).head()

### Inspecting Datatypes

In [ ]:
# Check if any of the columns marked as 'object' shouldn't be int or float instead
objs = df.columns[df.dtypes == object]
df[objs]

In [ ]:
df.Make

In [ ]:
df[objs[:2]]

Yuhk - full stops in the column names, let's get rid of those first.

In [ ]:
df.columns = [col.replace('.','') for col in df.columns]

In [ ]:
# Hmmm why is the cylinder not an integer dtype?
df.Cylinders

In [ ]:
# Let's try coercing it into an int datatype
try:
    [int(v) for v in df.Cylinders if type(v)]
except ValueError as e:
    print(e)

In [ ]:
# Ah, there's a row with a peculiar value
df[df.Cylinders == 'rotary']

In [ ]:
# Let's set the number of cylinders to 0
df.loc[df.Cylinders == 'rotary', 'Cylinders'] = 0

In [ ]:
# Change the Cylinder datatype
df.Cylinders = df.Cylinders.astype(int)
df.Cylinders.dtype

### Handling Missing Values

In [ ]:
# Check for missing values

# Create a helper function to catch columns with any missing values
# Logic : Check if there are any missing values, and sum them. True
# counts as '1' so that's why this elegant solution works! 
is_null = lambda col: sum(pd.isnull(df[col]))

# Logic : return (column name, missing value count) for each of the
# columns in the dataframe, _if_ there are any missing values
missing_values = [(col,is_null(col)) for col in df if is_null(col)]
missing_values

Two columns have missing values, let's take a closer look at the data and figure out why that's the case.

#### Rear.seat.room

In [ ]:
select = missing_values[0][0]
df[df[select].isnull()]

OK - so they are both sports cars. But let's sort by `Rear.seat.room` and see whether it's just that there _is_ no rear seat space, or whether the data is truly missing.

In [ ]:
df.sort_values('Rearseatroom')[['Manufacturer','Model','Type','Rearseatroom']]

There are no cars where the `Rear.seat.room` is set to zero, so after a bit of Googling, you'd find that indeed these two cars don't allow for passengers in the back, so we can set these values to zero.

In [ ]:
crit = df['Rearseatroom'].isnull()
df.loc[crit, 'Rearseatroom'] = 0

#### Luggage.room

Now let's inspect the missing values for `Luggage.room`

In [ ]:
select = missing_values[1][0]
df[df[select].isnull()]

Lot's of vans, and the two sports cars we just dealt with for missing Rear.seat.room values! Let's see how Vans 
are dealt with.

In [ ]:
df[df.Type == 'Van']

OK - turns out none of the vans are considered to have 'luggage room' , so let's also set ll these values to zero.

In [ ]:
crit = df['Luggageroom'].isnull()
df.loc[crit, 'Luggageroom'] = 0

Sanity check to make sure we can move on now.

In [ ]:
missing_values = [(col,is_null(col)) for col in df if is_null(col)]
len(missing_values)

### Split out design matrix and response vector

In [ ]:
# Let's build a model for MPG.city
y = df['MPGcity']
# y = df['MPG.highway']

# Lazy selection of all numeric columns without MPG
selection = df.select_dtypes(['int','float']).columns.difference(['MPGcity','MPGhighway'])
X = df[selection]
X.head()

### Feature Selection

Is there a potential relationship between the predictors and the response variable?

In [ ]:
from sklearn import feature_selection as fs

def f_regression_feature_selection(input, response):    
# use this against your feature matrix to determine p-values for
# each feature (we care about the second array it returns).
    return fs.univariate_selection.f_regression(input, response)    

#### ValueError?

If your attempt of running `f_regression_feature_selection` with a current `X` and `y` failed with the error:

```bash
ValueError: Input contains NaN, infinity or a value too large for dtype('float64').
```
this meant that you likely didn't properly clean out the missing values from your design matrix.

### Visual Exploration of Correlation

In [ ]:
data = pd.DataFrame(X,y).reset_index()

In [ ]:
g = sns.PairGrid(data, diag_sharey=False)
g.map_lower(sns.kdeplot, cmap="Blues_d")
g.map_upper(plt.scatter)
g.map_diag(sns.kdeplot, lw=3);

That gives us something to work with, but there's no clear slam dunk. also, various features appear to have a curvilinear relation with MPG.

### F Values

The "F value'' statistics tests the overall significance of the regression model.  Specifically, it tests the null hypothesis that all of the regression coefficients are equal to zero.  This tests the full model against a model with no variables and with the estimate of the dependent variable being the mean of the values of the dependent variable.  The F value is the ratio of the mean regression sum of squares divided by the mean error sum of squares.  Its value will range from zero to an arbitrarily large number.

The null hypothesis is rejected if the F ratio is large. Some analysts recommend ignoring the P values for the individual regression coefficients if the overall F ratio is not statistically significant, because of the problems caused by multiple testing. Here we'll reject the features if they are smaller than 10.



In [ ]:
# How many features don't meet the F test threshold?
sum(f_regression_feature_selection(X,y)[0] < 10)

In [ ]:
# Which column are we talking about?
select = f_regression_feature_selection(X,y)[0] < 10
X.columns[select]

The two columns we replaced the missing values! At this point I'm more than happy to drop them.

In [ ]:
# difference between the ones available and the ones we wish to drop
post_select = X.columns.difference(X.columns[select])

Xs = X[post_select]

In [ ]:
Xs.head()

### P values

In [ ]:
# How many features don't meet the F test threshold?
sum(f_regression_feature_selection(Xs,y)[1] > 0.05)

Great! All of our predictors are significant ( p < 0.05 ) in a univariate regression.

The homework asked to select featured based on their P value.

In [ ]:
# Sort the features based on their statistical significance 
ps = f_regression_feature_selection(Xs,y)[1]

p_score = zip(Xs.columns, ps)
ranked_p = sorted(p_score, key=lambda x:x[1])
ranked_p

Now, we don't have the full toolkit yet to guard against overfitting - we'll introduce this in another lesson - but this is how you would loop through the various features and see the scores.

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
# Let's first build univariate models to see how well each individual features performs
scores = []
for feat, score in ranked_p:
    est = Ridge()
    X = [[x] for x in Xs[feat]]
    est.fit(X,y)
    scores.append(est.score(X,y))

In [ ]:
# Drop of R^2 with strong to weakest features
cols = [k[0] for k in ranked_p]
pd.DataFrame(scores,index=cols).iplot(kind='bar')

The first 3 features seem to be reasonably powerful, after which the next 6 features are mediocre, and the remaining features aren't useful by themselves and will likely be dropped. 

In [ ]:
# Now let's build models which cummatively look well combinations of features performs
scores = []
feats = []
for feat, score in ranked_p:
    est = Ridge()
    feats.append(feat)
    if len(feats) == 1:
        X = [[x] for x in Xs[feat]]
    else:
        X = Xs[feats]
    est.fit(X, y)
    scores.append(est.score(X,y))

In [ ]:
# Drop of R^2 with strong to weakest features
# Drop of R^2 with strong to weakest features
# cols = [k[0] for k in ranked_p]
pd.DataFrame(scores,index=feats).iplot(kind='bar')

The first feature `Weight` is of course the most important, but the second and third features don't give us as much lift as we might have hoped for. This might have been due to the first three values being highly multicollinear. Let's inspect.

In [ ]:
correlation = Xs[[x[0] for x in ranked_p][:3]].corr()

In [ ]:
correlation.iplot(kind='heatmap', colorscale='spectral')

Compare this with the correlation plot of the whole dataset, and indeed! Apart from price correlating with min- and max-price, these are some of the most highly correlated features.

In [ ]:
f, ax = plt.subplots(figsize=(12, 12))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.corrplot(Xs, annot=False, sig_stars=False,
             diag_names=False, cmap=cmap, ax=ax)
f.tight_layout()

So, based on the P values, the most important features are

In [ ]:
p_features = [x[0] for x in ranked_p][:5]
p_features

Based on the the following plot, the feature which provide the largest improvement in R^2 are features [0,3,5], just out of interest, let's also build and score a model with only those features. 

In [ ]:
# Drop of R^2 with strong to weakest features
plt.plot(range(len(scores)), scores)

In [ ]:
handpicked_features = [ranked_p[x][0] for x in [0,3,5]]
print handpicked_features

In [ ]:
est = Ridge()
X = Xs[handpicked_features]
est.fit(X, y).score(X,y)

Even though the addition of these features was pivotal for the previous cummalative model, when just taken alone 

### Recursive feature elimination

We could have also decided to use [Recursive Feature Elimination](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)

Given an external estimator that assigns weights to features (e.g., the coefficients of a linear model), the goal of recursive feature elimination (RFE) is to select features by recursively considering smaller and smaller sets of features. First, the estimator is trained on the initial set of features and weights are assigned to each one of them. Then, features whose absolute weights are the smallest are pruned from the current set features. That procedure is recursively repeated on the pruned set until the desired number of features to select is eventually reached.

Since the RFE used our _coefficient size_ as a determinant of importance, we need to make sure we **standardise the data first**! This way the mean is always close to zero, and the standard deviation is one. 

In [ ]:
stand_Xs = (Xs - Xs.mean()) / Xs.std()

In [ ]:
from sklearn.feature_selection import RFE

# Create the RFE object and rank each features
est = Ridge()
rfe = RFE(estimator=est, n_features_to_select=1, step=1)

rfe.fit(stand_Xs, y)
ranking = rfe.ranking_

scores = zip(Xs.columns,ranking)
scores = sorted(scores, key=lambda x:x[1])

In [ ]:
# Feature Importance 
plt.figure(figsize=(16, 6))
sns.barplot(Xs.columns, 16 - ranking);

So, how did the automatic RFE do compared to our own more manual feature selection?

In [ ]:
rfe_features = [x[0] for x in scores][:5]
print 'P Value:', p_features
print 'RFE Value:', rfe_features
print 'Handpicked Values:', handpicked_features

In [ ]:
for feats in [handpicked_features, p_features, rfe_features]:
    est = Ridge()
    X = Xs[feats]
    print est.fit(X, y).score(X,y)

The RFE feature selection delivered the strongest model.

### Polynominal Features

In [ ]:
from patsy import dmatrix

We will expand on polynominal feature selection once we introduce cross-validation, but for now let's use Ridge Regression to prevent against overfitting. Let's continue with the features found by the RFE.

In [ ]:
rfe_features

In [ ]:
poly_features = []

In [ ]:
[poly_features.append(dmatrix('C('+ str(feat) +', Poly)', Xs)) for feat in rfe_features];

In [ ]:
poly_features

Let's build a new dataframe which has all the polynomial columns

In [ ]:
poly_featurs_123 = [poly[:, 1:4] for poly in poly_features]

In [ ]:
p = poly_featurs_123

dfs = [pd.DataFrame(p[x]) for x in range(5)]
dfs = pd.concat(dfs, axis=1)

We need to rewrite the columns names since they got lost in the transformation

In [ ]:
column_names = [name + '^' + str(e) for name in rfe_features for e in range(1,4)]
column_names

In [ ]:
dfs.columns = column_names

In [ ]:
dfs

Now, finally, let's build a model with our 5 features all with 3rd order polynomimials.

In [ ]:
est = Ridge()
print "R^2 of %.02f based on %s features" % (est.fit(dfs, y).score(dfs,y), len(dfs.columns))

The model didn't improve spectacularly, so until we are better equiped to evaluated our models, we ought to stick with the linear model.

#### PolynomialFeatures

Alternatively, you can use the `PolynomialFeatures` function from Scikit-Learn.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
poly = PolynomialFeatures(3)
poly_Xs = poly.fit_transform(Xs)

How many features did we create?

In [ ]:
len(poly_Xs) - len(Xs.columns)

Yep - close to 100 features. Just for kicks, how close to 1 could we push our R^2?

In [ ]:
est = Ridge()
print "R^2 of %.02f based on %s features" % (est.fit(poly_Xs, y).score(poly_Xs,y), len(poly_Xs))

Yep - perfect model - perfectly overfit that is! We'll see soon how to deal with this.